In [1]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [2]:
# Generate submechanism
from automol.graph import enum

from automech.species import Species

gen_mech = automech.from_smiles(
    spc_smis=["C1=C[CH]CC1", "C1=CC[CH]C1"],
    src_mech=par_mech,
)
#  - Additions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.o2_addition,
    rcts_=[None, "[O][O]"],
    spc_col_=Species.smiles,
    src_mech=par_mech,
)
#  - Eliminations
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.ho2_elimination,
    spc_col_=Species.smiles,
    src_mech=par_mech,
)
#  - QOOH formation
gen_mech = automech.enumerate_reactions(
    gen_mech, enum.ReactionSmarts.qooh_formation, src_mech=par_mech
)
#   - Ring-forming scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.qooh_ring_forming_scission,
    src_mech=par_mech,
    excl_rcts=automech.unstable_species_names(gen_mech),
)
#  - Ring-opening scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.ring_beta_scission,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech.species, fml="C*H*O2")],
    excl_rcts=automech.unstable_species_names(gen_mech),
)
#  - Beta scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.beta_scission,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech.species, fml="C*H*O2")],
    excl_rcts=automech.unstable_species_names(gen_mech),
    repeat=2,
)
# Dropping these for now because they are failing (Y70-73, 76-79)
# #  - Vinyl beta scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech2,
#     enum.ReactionSmarts.vinyl_beta_scission,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
#  - Instability products
gen_mech = automech.enumerate_products(
    gen_mech,
    enum.ReactionSmarts.qooh_instability,
    src_mech=par_mech,
    rcts_=[automech.species.names(gen_mech.species, fml="C*H*O2")],
)
#  - Drop reactions:
gen_mech = automech.drop_reactions_by_smiles(
    # >> Beta elimination: all TSs converge to VDW SPs:
    gen_mech, rxn_smis=["[O]OC1C=CCC1>>C=1=CCCC=1.[O]O"]
)
automech.display(gen_mech, height="1000px")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Expand stereochemistry
ste_mech, *_, gen_mech = workflow.expand_stereo(
    mech=gen_mech, tag=tag, root_path=root_path
)

#  - Drop stereo-specific reactions
ste_mech = automech.drop_reactions_by_smiles(
    # >> Cis ring-opening scission
    ste_mech,
    rxn_smis=[
        r"OO[C@H]1[CH]C=CC1>>[CH2]/C=/C\C=\C/OO",
        r"OO[C@@H]1[CH]C=CC1>>[CH2]/C=/C\C=\C/OO",
    ],
)


Expanding stereochemistry...


  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]


Stereoexpansion errors:


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Prepare calculation
workflow.prepare_calculation(
    gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
)


Sorting mechanism...

Writing mechanism...
../data/1e_D_r-o2_v0_gen.json
../data/1e_D_r-o2_v0_ste.json
../data/mechanalyzer/1e_D_r-o2_v0_ste.dat
../data/mechanalyzer/1e_D_r-o2_v0_ste.csv


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

In [5]:
# # Gather statistics
# workflow.gather_statistics(tag=tag, root_path=root_path)

In [6]:
# # Prepare simulation
# workflow.prepare_simulation(tag=tag, root_path=root_path)
# workflow.prepare_simulation_species(tag=tag, root_path=root_path)

In [7]:
# # Check rates
# workflow.plot_rates(tag=tag, root_path=root_path)

In [8]:
# # Plot simulation
# chart_dct = workflow.plot_simulation(
#     tag=tag,
#     x_col="O2_molecules",
#     root_path=root_path,
#     line_source_=["Hill", "Lokachari"],
#     point_source="Experiment",
#     control=False,
# )
# mech = automech.io.read(
#     p_.full_calculated_mechanism(tag, "json", path=p_.data(root_path))
# )
# for name, chart in chart_dct.items():
#     automech.display_species(mech, [name])
#     chart.show()